<a href="https://colab.research.google.com/github/pushna99/CS682-Data-Center-NO2/blob/main/CS682_Project_Pushna_Version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy import distance

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

# Get the contents of NASA Timeseries files through URL
!wget https://so2.gsfc.nasa.gov/no2/pix/time_series/OMNO2_Timeseries_AllCities.csv -q
data=pd.read_csv('OMNO2_Timeseries_AllCities.csv',index_col=False, on_bad_lines='skip',skiprows=7)
df = pd.DataFrame(data)

In [ ]:
sum_med = df.groupby('City')['Median'].mean()
print(sum_med)

with open('OMNO2_Timeseries_AllCities.csv','r') as f_in:
    f = csv.reader(f_in)
    rows=df.groupby(['City','Country','LonCenter','LatCenter']).agg({'Median': ['mean']})
    rows.columns = ['Average NO2']
    rows = rows.reset_index()

geolocator = Nominatim(user_agent='myapplication')
for index, row in rows.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    rows.loc[index,"LatCenter"]=location.latitude
    rows.loc[index,"LonCenter"]=location.longitude


#Authorize access to google sheets and fetch the nasa-cities file into a dataframe
creds, _ = default()
gc = gspread.authorize(creds)
nasa_cities_worksheet = gc.open('nasa-cities').sheet1
rows = nasa_cities_worksheet.get_all_values()
nasa_cities = pd.DataFrame.from_records(rows)
nasa_cities.columns = nasa_cities.iloc[0]
nasa_cities = nasa_cities[1:]
nasa_cities = nasa_cities.rename(columns={'City': 'City Name','LonCenter':'Longitude','LatCenter':'Latitude'})
nasa_cities.reset_index(drop=True, inplace=True)

City
Aberdeen     -0.241590
Abidjan      -0.289164
AbuDhabi      2.574397
Abuja         1.005829
Accra        -1.289991
               ...    
Yangoon     -13.736218
Yaounde      -0.751641
Yerevan       1.201822
Zhengzhou     6.617612
Zurich        1.628643
Name: Median, Length: 316, dtype: float64


In [ ]:
#Read contents of lat-lon file
latlong = gc.open('lat-lon').sheet1
rows = latlong.get_all_values()
dfll = pd.DataFrame.from_records(rows)
# Convert first column to header
dfll.columns = dfll.iloc[0]
dfll = dfll[1:]

#read contents of input file containing customer cities
cityname = gc.open('input').sheet1
rows = cityname.get_all_values()
dfcity = pd.DataFrame.from_records(rows)
# Convert first column to header
dfcity.columns = dfcity.iloc[0]
dfcity = dfcity[1:]

#Add two new columns "Latitude" and "Longitude"
dfcity["Latitude"] = pd.NaT
dfcity["Longitude"] = pd.NaT
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfcity.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    dfcity.loc[index,"Latitude"]=location.latitude
    dfcity.loc[index,"Longitude"]=location.longitude

#Drop NaT values and update the lat-lon file
dfcity=dfcity.dropna()
dfcity.update([dfcity.columns.values.tolist()] + dfcity.values.tolist())

customer_cities = pd.concat([dfcity,dfll], axis = 0).drop_duplicates().reset_index(drop=True)

In [ ]:
nasa_cities = nasa_cities.apply(pd.to_numeric, errors='ignore')
customer_cities = customer_cities.apply(pd.to_numeric, errors='ignore')
#NO2data = pd.merge(customer_cities,nasa_cities , on=['Longitude','Latitude'])
customer_cities["Average NO2"] = pd.NaT

# Check the nearest city based on latitude and longitude values
for index, row in customer_cities.iterrows():
  min_distance=float('inf')
  for nindex, nrow in nasa_cities.iterrows():
    if(distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles < min_distance):
      min_distance = distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles
      min_index = nindex
  customer_cities.loc[index,'Average NO2'] = nasa_cities['Average NO2'].values[min_index]

In [ ]:
#Creating a new Average NO2 data in the output folder
average_no2_data = gc.create('average_no2_data').sheet1
average_no2_data.update([customer_cities.columns.values.tolist()] + customer_cities.values.tolist())

{'spreadsheetId': '1ThRkPX9sAHfWlWcxZPlPNUyiaJ3tUnOtEmuKbRxqlzg',
 'updatedCells': 535,
 'updatedColumns': 5,
 'updatedRange': 'Sheet1!A1:E107',
 'updatedRows': 107}